In [6]:
from scipy import special
import numpy as np
import matplotlib.pyplot as plt

In [59]:
## object class for node
class node():
    """the class for each node"""
    
    def __init__(self, left, right, data):
        self.l = left
        self.r = right
        if(left == None and right == None):
            self.n = 1
        else:
            self.n = left.n + right.n
        self.data = data

In [47]:
## function to calculate the posterior probability
def p_hyp1(dataset, a):
    # extract the number of features and the total number of data
    k = dataset.shape[1]
    N = dataset.shape[0]
    
    # part I
    p1 = 1
    for i in range(N):
        comp = special.gamma(np.sum(dataset[i, :])+1) / np.prod(special.gamma(dataset[i, :]+1))
        p1 = p1 * comp
    
    # part II
    # iterate to calculate the probability
    p2 = p1 * special.gamma(np.sum(a)) / special.gamma(np.sum(dataset) + np.sum(a))
    for j in range(k):
        comp = special.gamma(a[j] + np.sum(dataset[:, j])) / special.gamma(a[j])
        p2 = p2 * comp

    return p2

In [48]:
## function to calculate the d
def get_d(node, a):
    if node.l == None and node.r == None:
        return a
    else:
        return a*special.gamma(node.n) + get_d(node.l, a)*get_d(node.r, a)

In [49]:
## function to calculate the weight or pi
def get_pi(node, a):
    dk = get_d(node, a)
    pi_k = a*special.gamma(node.n)/dk
    return pi_k

In [50]:
# test function
p_hyp1(data, a)

6.6140820029103093e-13

In [51]:
# test functions
n1 = node(left = None, right = None)
n2 = node(left = None, right = None)
n3 = node(left = n1, right = n2)
get_pi(n3,1)

0.5

In [53]:
## test dataset
data = np.random.randint(0,4, size=(5,5))
a = [1,1,1,1,1]
data

array([[0, 2, 0, 1, 1],
       [2, 3, 2, 3, 2],
       [3, 0, 3, 0, 0],
       [3, 2, 3, 2, 0],
       [0, 1, 1, 1, 3]])

In [60]:
# get dk
def get_dk(node, a):
    post = p_hyp1(node_new.data, np.repeat(a, data.shape[1]))
    pi = get_pi(node, a)
    if node.l == None and node.r == None:
        return  pi * post
    else:
        return  pi * post + (1-pi) * get_dk(node.l) * get_dk(node.r)

In [56]:
node_list = []
for i in range(data.shape[0]):
    node_list.append(node(None, None, data[i,:]))

c = data.shape[0]
a = 1

while c > 1:
    flag = False
    for i in range(len(node_list)):
        for j in range(i+1, len(node_list)):
            node_new = node(node_list[i], node_list[j], np.concatenate((node_list[i].data, node_list[j].data), axis=0))
            pi_k = get_pi(node_new, a)
            p_hyp1(node_new.data, np.repeat(a, data.shape[1]))
            p_dk = get_dk(node_new)
            rk = pi_k * p_hyp1 / p_dk
            if rk > 0.5:
                node_list.append(node_new)
                node_list = node_list[:i] + node_list[i+1:j] + node_list[j:]
                c = c - 1
                flag = True
                next
    if(!flag):
        c = 1
    
    

In [79]:
a = list(range(10))
a[:2] + a[4:6] + a[9:]

[0, 1, 4, 5, 9]

In [82]:
list(range(0,3))

[0, 1, 2]